<a href="https://colab.research.google.com/github/abhaygangwar3/Ankii-s-Portfolio/blob/master/ConvXGB_method_to_classify_the_OPG_images_as_per_the_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ConvXGB method to classify the OPG images as per the gender**

**Step 1**


In [120]:
import numpy as np
from sklearn.metrics import accuracy_score
import xgboost as xgb
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical

In [121]:
# Step 1: Load the dataset from separate .npz files
def load_data(npz_path):
    data = np.load(npz_path)
    X = data['X']
    y = data['y']
    return X, y

In [122]:
# Paths to your .npz files (replace with the actual paths)
train_npz_path = '/content/drive/MyDrive/Train.npz'
val_npz_path = '/content/drive/MyDrive/Val.npz'
test_npz_path = '/content/drive/MyDrive/Test.npz'

In [123]:
data_train = np.load(train_npz_path)
data_test = np.load(val_npz_path)
data_val = np.load(test_npz_path)


In [131]:
# Load train, validation, and test sets
images_train = data_train['images']
labels_train = data_train['labels']
images_val = data_val['images']
labels_val = data_val['labels']
images_test = data_test['images']
labels_test = data_test['labels']

In [100]:
print(images_train.shape)
print(labels_train.shape)
print(images_val.shape)
print(labels_val.shape)
print(images_test.shape)
print(labels_test.shape)

(562, 360, 120, 3)
(562,)
(12, 120, 360, 3)
(12,)
(106, 360, 120, 3)
(106,)


In [132]:
# Step 2: Normalize the images (assuming they are grayscale)
images_train = images_train / 255.0
images_val = images_val / 255.0
images_test = images_test / 255.0

In [126]:
import cv2
import numpy as np

# Convert 3-channel (RGB) images to grayscale
images_train_gray = np.array([cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_RGB2GRAY) for img in images_train]) # Change the image type to np.uint8
images_val_gray = np.array([cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_RGB2GRAY) for img in images_val]) # Change the image type to np.uint8
images_test_gray = np.array([cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_RGB2GRAY) for img in images_test]) # Change the image type to np.uint8

# Now images are grayscale, we need to reshape them to have a single channel (for CNN input)
# Add an extra dimension at the end (channels = 1 for grayscale images)
images_train_gray = images_train_gray.reshape(-1, images_train.shape[1], images_train.shape[2], 1)
images_val_gray = images_val_gray.reshape(-1, images_val.shape[1], images_val.shape[2], 1)
images_test_gray = images_test_gray.reshape(-1, images_test.shape[1], images_test.shape[2], 1)

print("New shape of images_train:", images_train_gray.shape)  # Should be (562, 360, 120, 1)
print("New shape of images_val:", images_val_gray.shape)      # Should be similar for validation
print("New shape of images_test:", images_test_gray.shape)    # Similar for test data

New shape of images_train: (562, 360, 120, 1)
New shape of images_val: (12, 120, 360, 1)
New shape of images_test: (106, 360, 120, 1)


In [127]:
from keras.utils import to_categorical

# Convert the labels to one-hot encoding
labels_train_one_hot = to_categorical(labels_train, num_classes=2)
labels_val_one_hot = to_categorical(labels_val, num_classes=2)
labels_test_one_hot = to_categorical(labels_test, num_classes=2)

print(f"Shape of labels_train_one_hot: {labels_train_one_hot.shape}")  # Should be (562, 2)


Shape of labels_train_one_hot: (562, 2)


In [128]:
# Step 3: Reshape the data for CNN input (assuming grayscale images)
img_size = (images_train.shape[1], images_train.shape[2])  # Assuming (samples, height, width) shape
images_train = images_train.reshape(-1, img_size[0], img_size[1], 1)
images_val = images_val.reshape(-1, img_size[0], img_size[1], 1)
images_test = images_test.reshape(-1, img_size[0], img_size[1], 1)

In [105]:
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))  # Dropout to prevent overfitting

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))  # Dropout after every MaxPooling layer

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))  # Dropout after every Conv block

    # Adding one more convolutional block for higher capacity
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))  # Dropout before final dense layers

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))  # Increased the size of the dense layer
    model.add(Dropout(0.5))  # Dropout before final dense layers
    model.add(Dense(2, activation='softmax'))

    return model



In [106]:
print(f'images_train shape: {images_train.shape}')
print(f'labels_train shape: {labels_train.shape}')
print(f'images_val shape: {images_val.shape}')
print(f'labels_val shape: {labels_val.shape}')

images_train shape: (1686, 360, 120, 1)
labels_train shape: (562,)
images_val shape: (36, 360, 120, 1)
labels_val shape: (12,)


In [107]:
# Create CNN model
cnn_model = create_cnn_model((img_size[0], img_size[1], 1))
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [108]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# # Adjust the augmentation settings
# datagen = ImageDataGenerator(
#     rotation_range=5,      # Reduced rotation range
#     width_shift_range=0.05,  # Smaller width shift
#     height_shift_range=0.05, # Smaller height shift
#     zoom_range=0.05,         # Smaller zoom
#     horizontal_flip=True,   # Horizontal flip should help in gender classification
#     fill_mode='nearest'     # Default fill mode
# )


In [109]:
# Step 5: Train the CNN model on the train set and validate on the validation set
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

cnn_model.fit(
    images_train_gray, labels_train_one_hot,
    validation_data=(images_val_gray, labels_val_one_hot),
    epochs=50, callbacks=[reduce_lr, early_stopping]  # Train for a few epochs without augmentation
)

# Then introduce augmentation gradually
# cnn_model.fit(
#     datagen.flow(images_train_gray, labels_train_one_hot, batch_size=32),
#     validation_data=(images_val_gray, labels_val_one_hot),
#     epochs=20,  # Continue with mild augmentation,
#     callbacks=[reduce_lr, early_stopping]
# )

# cnn_model.fit(images_train_gray, labels_train_one_hot, epochs=20, batch_size=32, validation_data=(images_val_gray, labels_val_one_hot),
#               callbacks=[reduce_lr, early_stopping])

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 79s 4s/step - accuracy: 0.5829 - loss: 0.6833 - val_accuracy: 0.7500 - val_loss: 0.6667 - learning_rate: 0.0010
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 80s 4s/step - accuracy: 0.6408 - loss: 0.6578 - val_accuracy: 0.7500 - val_loss: 0.6555 - learning_rate: 0.0010
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 74s 4s/step - accuracy: 0.6402 - loss: 0.6341 - val_accuracy: 0.5833 - val_loss: 0.6477 - learning_rate: 0.0010
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.6368 - loss: 0.6216 - val_accuracy: 0.7500 - val_loss: 0.6380 - learning_rate: 0.0010
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 83s 4s/step - accuracy: 0.6591 - loss: 0.5820 - val_accuracy: 0.6667 - val_loss: 0.6922 - learning_rate: 0.0010
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.7126 - loss: 0.5669 - val_accuracy: 0.5000 - val_loss: 0.7657 - learning_rate: 0.0010
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.7178 - loss: 0.5148 - val_accuracy: 

In [110]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Create an image data generator with augmentation
# datagen = ImageDataGenerator(
#     rotation_range=10,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # Fit the generator to your training data
# datagen.fit(images_train_gray)

# Use the augmented data generator in model fitting
# cnn_model.fit(
#     datagen.flow(images_train_gray, labels_train_one_hot, batch_size=32),
#     validation_data=(images_val_gray, labels_val_one_hot),
#     epochs=20
# )

In [111]:
# Step 6: Remove the last layer from CNN to extract features
cnn_model.pop()  # Remove the final dense layer to use CNN as a feature extractor

<Dense name=dense_7, built=True>

In [112]:
# Extract features from train, validation, and test sets
train_features = cnn_model.predict(images_train_gray)
images_val_gray = images_val_gray.reshape((12, 360, 120, 1))
val_features = cnn_model.predict(images_val_gray)
test_features = cnn_model.predict(images_test_gray)

18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step


In [113]:
# Step 7: Train XGBoost Classifier on Extracted Features
# xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=5, random_state=42)

In [114]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Set up the parameter grid for XGBoost
param_grid = {
    'n_estimators': [200],
    'max_depth': [5],
    'learning_rate': [0.01],
    'subsample': [0.8],
    'colsample_bytree': [0.8]
}

# Initialize the model
xgb_model = xgb.XGBClassifier()

# Grid search for best parameters
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(train_features, labels_train)

# Best model after grid search
best_model = grid_search.best_estimator_
print(f"Best XGBoost parameters: {grid_search.best_params_}")

# Evaluate on test set
y_pred = best_model.predict(test_features)
accuracy = accuracy_score(labels_test, y_pred)
print(f"Test Accuracy after tuning: {accuracy:.4f}")


Best XGBoost parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}
Test Accuracy after tuning: 0.5943


In [115]:
# Train XGBoost on CNN features
# xgb_model.fit(train_features, labels_train)

In [116]:
# Step 8: Evaluate the Model on the Test Data
# y_pred = xgb_model.predict(test_features)
# accuracy = accuracy_score(labels_test, y_pred)
# print(f"Test Accuracy: {accuracy:.4f}")

## Using different techniques to increase accuracy



In [118]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

# Load VGG16 pre-trained model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(360, 120, 3))

# Freeze the base model layers (so we don't train them initially)
base_model.trainable = False

# Create the model
model = Sequential()

# Add the pre-trained base model
model.add(base_model)

# Add custom layers on top of it
model.add(Flatten())
model.add(Dense(128, activation='relu'))  # Dense layer with 128 units
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Final classification layer (1 unit for binary classification)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

# Load VGG16 pre-trained model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(360, 120, 3))

# Freeze the base model layers (so we don't train them initially)
base_model.trainable = False

# Create the model
model = Sequential()

# Add the pre-trained base model
model.add(base_model)

# Add custom layers on top of it
model.add(Flatten())

# You can't access output_shape before the model has been built
# Call model.build() or model.predict() to define the output shape
model.build(input_shape=(None, 360, 120, 3))

# Now you can print the output shape
print(model.output_shape)

model.add(Dense(128, activation='relu'))  # Dense layer with 128 units
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Final classification layer (1 unit for binary classification)

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

images_val_mod = images_val.reshape((12, 360, 120, 3))

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Assuming images_train, labels_train, images_val, and labels_val are already loaded
train_generator = datagen.flow(images_train, labels_train, batch_size=32)

# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=(images_val_mod, labels_val))

(None, 16896)
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 347s 19s/step - accuracy: 0.4450 - loss: 0.8977 - val_accuracy: 0.7500 - val_loss: 0.6702
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 366s 18s/step - accuracy: 0.5054 - loss: 0.7574 - val_accuracy: 0.6667 - val_loss: 0.6785
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - accuracy: 0.5009 - loss: 0.7124 

In [ ]:
# Unfreeze some of the deeper layers of the base model
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Re-compile the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Continue training (fine-tuning)
history = model.fit(train_generator,
                    epochs=5,
                    validation_data=(images_val, labels_val))


In [ ]:
test_loss, test_accuracy = model.evaluate(images_test, labels_test)
print(f'Test Accuracy: {test_accuracy:.4f}')
